In [2]:
import pandas as pd
import ast
import os

# Function to safely evaluate the string dictionary
def safe_eval_dict(details_text):
    try:
        # Convert string representation of dictionary to actual dictionary
        return ast.literal_eval(details_text)
    except (ValueError, SyntaxError):
        return {}

# Function to process a single CSV
def process_csv(file_path):
    # Load the CSV
    df = pd.read_csv(file_path)

    # Apply the safe_eval_dict function to the 'details_text' column
    df['details_dict'] = df['details_text'].apply(safe_eval_dict)

    # Normalize the 'details_dict' column to individual columns, filling missing values with NaN
    details_df = pd.json_normalize(df['details_dict'])

    # Concatenate the original dataframe with the new columns
    df_expanded = pd.concat([df.drop(columns=['details_text', 'details_dict']), details_df], axis=1)

    # Save the resulting DataFrame to a new CSV file (appending '_expanded' to the original file name)
    new_file_path = os.path.splitext(file_path)[0] + '_expanded.csv'
    df_expanded.to_csv(new_file_path, index=False)

    print(f"CSV expanded successfully: {new_file_path}")

# List of CSV file paths to process
csv_files = [
    'amsterdam.csv',
    'barcelona.csv',
    'rome.csv',
    'dubai.csv',
    'paris.csv'
]


# Loop over the list of CSVs and process each one
for csv_file in csv_files:
    process_csv(csv_file)


CSV expanded successfully: amsterdam_expanded.csv
CSV expanded successfully: barcelona_expanded.csv
CSV expanded successfully: rome_expanded.csv
CSV expanded successfully: dubai_expanded.csv
CSV expanded successfully: paris_expanded.csv


In [3]:
import pandas as pd
import os

def process_and_filter_csv(file_path, columns_to_keep):
    """
    Processes a single CSV, keeping only the specified columns and dropping the rest.

    :param file_path: The path to the CSV file to process.
    :param columns_to_keep: List of column names to keep in the DataFrame.
    """
    if not os.path.isfile(file_path):
        print(f"Warning: {file_path} does not exist and will be skipped.")
        return None
    
    try:
        # Load the CSV
        df = pd.read_csv(file_path, low_memory=False)
        print(f"Successfully read {file_path} with {len(df)} records.")
        
        # Keep only the specified columns
        df_filtered = df[columns_to_keep]

        # Save the filtered CSV with '_filtered' appended to the original file name
        filtered_file_path = os.path.splitext(file_path)[0] + '_filtered.csv'
        df_filtered.to_csv(filtered_file_path, index=False)
        
        print(f"Filtered CSV saved: {filtered_file_path}")
        
        return df_filtered

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

if __name__ == "__main__":
    # List of CSV filenames to process
    csv_files = [
        'amsterdam_expanded.csv',
        'barcelona_expanded.csv',
        'rome_expanded.csv',
        'dubai_expanded.csv',
        'paris_expanded.csv'
    ]

    # List of columns to keep
    columns_to_keep = [
        'location',
        'category',
        'place_id',
        'place_name',
        'reviews_text',
        'address',
        'international_phone_number',
        'lat',
        'lng',
        'polarity',
        'website'
    ]

    # Process each CSV file and filter columns
    for csv_file in csv_files:
        process_and_filter_csv(csv_file, columns_to_keep)


Successfully read amsterdam_expanded.csv with 20260 records.
Filtered CSV saved: amsterdam_expanded_filtered.csv
Successfully read barcelona_expanded.csv with 81950 records.
Filtered CSV saved: barcelona_expanded_filtered.csv
Successfully read rome_expanded.csv with 156602 records.
Filtered CSV saved: rome_expanded_filtered.csv
Successfully read dubai_expanded.csv with 177539 records.
Filtered CSV saved: dubai_expanded_filtered.csv
Successfully read paris_expanded.csv with 56529 records.
Filtered CSV saved: paris_expanded_filtered.csv


In [2]:
import pandas as pd
import pickle
import os
import ast

def combine_csv_files(csv_filenames, pickle_filename):
    """
    Combines multiple CSV files into a single Pandas DataFrame, removes rows with empty 'reviews_text',
    and saves the cleaned DataFrame as a pickle file.

    :param csv_filenames: List of CSV file paths to be combined.
    :param pickle_filename: The name of the output pickle file.
    :return: Cleaned Pandas DataFrame.
    """
    data_frames = []

    for file in csv_filenames:
        if not os.path.isfile(file):
            print(f"Warning: {file} does not exist and will be skipped.")
            continue
        try:
            # Add low_memory=False to avoid the DtypeWarning
            df = pd.read_csv(file, low_memory=False)
            print(f"Successfully read {file} with {len(df)} records.")
            data_frames.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if not data_frames:
        raise ValueError("No valid CSV files were provided.")

    # Concatenate all DataFrames into one
    combined_df = pd.concat(data_frames, ignore_index=True)
    print(f"Combined DataFrame has {len(combined_df)} records and {len(combined_df.columns)} columns.")

    # Optional: Parse complex fields from strings to Python objects
    for column in ['reviews_text', 'details_text']:
        if column in combined_df.columns:
            combined_df[column] = combined_df[column].apply(parse_complex_field)
            print(f"Parsed column: {column}")

    # --------------------- New Code Starts Here --------------------- #
    # Remove all rows where 'reviews_text' is empty or NaN
    if 'reviews_text' in combined_df.columns:
        initial_count = len(combined_df)
        
        # Remove rows where 'reviews_text' is NaN
        combined_df = combined_df.dropna(subset=['reviews_text'])
        
        # Define a function to check if 'reviews_text' is empty or contains only empty strings
        def is_reviews_text_empty(reviews):
            if isinstance(reviews, list):
                # Check if the list is empty
                if not reviews:
                    return True
                # Check if all elements in the list are empty strings or contain only whitespace
                return all(isinstance(review, str) and review.strip() == '' for review in reviews)
            # If not a list, consider it non-empty (or handle accordingly)
            return False
        
        # Apply the function to identify empty 'reviews_text'
        empty_reviews_mask = combined_df['reviews_text'].apply(is_reviews_text_empty)
        num_empty = empty_reviews_mask.sum()
        
        # Remove rows where 'reviews_text' is empty
        combined_df = combined_df[~empty_reviews_mask]
        
        final_count = len(combined_df)
        removed_rows = initial_count - final_count
        print(f"Removed {removed_rows} rows with empty 'reviews_text'. Remaining records: {final_count}")
    else:
        print("Column 'reviews_text' does not exist. No rows removed based on 'reviews_text'.")
    # --------------------- New Code Ends Here --------------------- #

    # Save the cleaned DataFrame as a pickle file
    try:
        with open(pickle_filename, 'wb') as pkl_file:
            pickle.dump(combined_df, pkl_file)
        print(f"Data successfully saved to {pickle_filename}")
    except Exception as e:
        print(f"Error saving pickle file: {e}")
        raise e

    return combined_df

def parse_complex_field(field):
    """
    Parses a string representation of a list or dictionary into actual Python objects.

    :param field: String representation of the field.
    :return: Parsed Python object (list or dict) or the original value if parsing fails.
    """
    if pd.isnull(field) or field == '':
        return None
    try:
        # Attempt to parse the field using ast.literal_eval
        parsed_field = ast.literal_eval(field)
        return parsed_field
    except (ValueError, SyntaxError):
        # If parsing fails, return the original string
        return field

def load_collection_data_frames(filename='collection_data_frames.pkl'):
    """
    Loads the pickled DataFrame from the specified file.

    :param filename: Path to the pickle file.
    :return: Loaded Pandas DataFrame.
    """
    try:
        with open(filename, 'rb') as file:
            data_frames = pickle.load(file)
        print(f"Data successfully loaded from {filename}")
        return data_frames
    except Exception as e:
        print(f"Error loading pickle file: {e}")
        raise e

if __name__ == "__main__":
    # Define the list of CSV filenames
    csv_files = [
        'Translated/amsterdam_translated.csv',
        'Translated/barcelona_translated.csv',
        'Translated/dubai_translated.csv',
        'Translated/paris_translated.csv',
        'Translated/rome_translated.csv'
    ]

    # Define the output pickle file name
    output_pickle = 'location_data_frames.pkl'

    # Combine CSV files and save as pickle
    location_data_frames = combine_csv_files(csv_files, output_pickle)

    # Optional: Display the first few rows of the combined DataFrame
    print("\nFirst 5 records of the combined DataFrame:")
    print(location_data_frames.head())


Successfully read Translated/amsterdam_translated.csv with 20260 records.
Successfully read Translated/barcelona_translated.csv with 81950 records.
Successfully read Translated/dubai_translated.csv with 177539 records.
Successfully read Translated/paris_translated.csv with 56529 records.
Successfully read Translated/rome_translated.csv with 156602 records.
Combined DataFrame has 492880 records and 11 columns.
Parsed column: reviews_text
Removed 375330 rows with empty 'reviews_text'. Remaining records: 117550
Data successfully saved to location_data_frames.pkl

First 5 records of the combined DataFrame:
    location       category  place_id                              place_name  \
1  Amsterdam  accommodation    223771                          American Hotel   
4  Amsterdam  accommodation    223829  Amsterdam Bed and Breakfast CityCenter   
5  Amsterdam  accommodation    223818                              BLUE TOWER   
6  Amsterdam  accommodation    223776              Hotel Espresso 

In [3]:
import pandas as pd
import pickle
import re

def load_collection_data_frames(filename='collection_data_frames.pkl'):
    """
    Loads the pickled DataFrame from the specified file.

    :param filename: Path to the pickle file.
    :return: Loaded Pandas DataFrame.
    """
    try:
        with open(filename, 'rb') as file:
            data_frames = pickle.load(file)
        print(f"Data successfully loaded from {filename}")
        return data_frames
    except Exception as e:
        print(f"Error loading pickle file: {e}")
        raise e

def parse_condition(cond_str, column):
    """
    Parses a condition string and returns a lambda function that applies the condition to a DataFrame column.

    Supported condition formats:
    - "x<value"
    - "x<=value"
    - "x>value"
    - "x>=value"
    - "x==value"
    - "x!=value"
    - "value1<x<value2"
    - "value1<=x<=value2"
    - "value1<x<=value2"
    - "value1<=x<value2"

    :param cond_str: Condition string (e.g., "0<x<50.4", "x<25").
    :param column: Column name to apply the condition on.
    :return: A Pandas Series representing the condition mask.
    """
    # Remove any spaces
    cond_str = cond_str.replace(" ", "")
    
    # Patterns for different condition types
    pattern_double = re.compile(r'^([<>]=?)(x)([<>]=?)([\d\.]+)$')
    pattern_between = re.compile(r'^([\d\.]+)([<>]=?)(x)([<>]=?)([\d\.]+)$')
    pattern_single = re.compile(r'^(x)([<>]=?)([\d\.]+)$')
    pattern_single_reverse = re.compile(r'^([\d\.]+)([<>]=?)(x)$')
    
    # Initialize mask as None
    mask = None
    
    # Check for between conditions like "0<x<50.4" or "10<=x<=20"
    match_between = re.match(r'^([\d\.]+)([<>]=?)(x)([<>]=?)([\d\.]+)$', cond_str)
    if match_between:
        val1, op1, _, op2, val2 = match_between.groups()
        val1 = float(val1)
        val2 = float(val2)
        if op1 in ['<', '<=']:
            condition1 = f"{column} {op1} {val2}"
        else:
            condition1 = f"{column} {op1} {val1}"
        
        if op2 in ['<', '<=']:
            condition2 = f"{column} {op2} {val2}"
        else:
            condition2 = f"{column} {op2} {val1}"
        
        # Apply both conditions
        mask = (df[column] > val1) if 'x>' in cond_str else (df[column] >= val1) if 'x>=' in cond_str else (df[column] < val2) if 'x<' in cond_str else (df[column] <= val2)
        # Alternatively, apply both conditions
        mask = (df[column] > val1) & (df[column] < val2)
        return mask
    
    # Check for single conditions like "x<25", "x>=10"
    match_single = re.match(r'^(x)([<>]=?)([\d\.]+)$', cond_str)
    if match_single:
        _, operator, value = match_single.groups()
        value = float(value)
        if operator == '<':
            mask = df[column] < value
        elif operator == '<=':
            mask = df[column] <= value
        elif operator == '>':
            mask = df[column] > value
        elif operator == '>=':
            mask = df[column] >= value
        elif operator == '==':
            mask = df[column] == value
        elif operator == '!=':
            mask = df[column] != value
        return mask
    
    # Check for reversed single conditions like "10<x"
    match_single_rev = re.match(r'^([\d\.]+)([<>]=?)(x)$', cond_str)
    if match_single_rev:
        value, operator, _ = match_single_rev.groups()
        value = float(value)
        if operator == '<':
            mask = df[column] > value
        elif operator == '<=':
            mask = df[column] >= value
        elif operator == '>':
            mask = df[column] < value
        elif operator == '>=':
            mask = df[column] <= value
        elif operator == '==':
            mask = df[column] == value
        elif operator == '!=':
            mask = df[column] != value
        return mask
    
    # If no pattern matches, raise an error
    raise ValueError(f"Unsupported condition format: {cond_str}")

def search_database(df, search_criteria):
    """
    Searches the DataFrame based on the provided search criteria.

    :param df: Pandas DataFrame to search.
    :param search_criteria: Dictionary containing search conditions and columns to return.
                            Example:
                            {
                                'location': 'Amsterdam',
                                'category': 'Accommodation',
                                'lat': '0<x<50.4',
                                'lng': 'x<25',
                                'columns': 'all'
                            }
    :return: Filtered Pandas DataFrame based on the search criteria.
    """
    mask = pd.Series([True] * len(df))
    
    # Iterate through each key in the search criteria
    for key, value in search_criteria.items():
        if key == 'columns':
            continue  # Handle columns later
        if key not in df.columns:
            print(f"Warning: Column '{key}' does not exist in the DataFrame and will be ignored.")
            continue
        
        if isinstance(value, str) and 'x' in value:
            try:
                condition_mask = parse_condition(value, key)
                mask &= condition_mask
            except ValueError as ve:
                print(f"Error parsing condition for column '{key}': {ve}")
                continue
        else:
            # Exact match
            mask &= df[key] == value
    
    # Apply the mask to filter the DataFrame
    filtered_df = df[mask]
    
    # Handle the 'columns' key
    columns_to_return = search_criteria.get('columns', 'all')
    if columns_to_return != 'all':
        if isinstance(columns_to_return, list):
            # Validate columns
            valid_columns = [col for col in columns_to_return if col in df.columns]
            invalid_columns = set(columns_to_return) - set(valid_columns)
            if invalid_columns:
                print(f"Warning: The following columns are invalid and will be ignored: {invalid_columns}")
            if valid_columns:
                filtered_df = filtered_df[valid_columns]
            else:
                print("Warning: No valid columns specified. Returning all columns.")
        else:
            print("Warning: 'columns' should be a list or 'all'. Returning all columns.")
    
    return filtered_df

# Load the DataFrame from the pickle file
df = load_collection_data_frames('location_data_frames.pkl')

# Example Search Criteria
# search_criteria_1 = {
#     'location': 'Amsterdam',
#     'category': 'accommodation',
#     'lat': '50<x<53',
#     'lng': '4<x<5',
#     'columns': 'all'
# }

search_criteria_2 = {
    'location': 'Barcelona',
    'columns': ['place_name', 'address', 'website']
}

# search_criteria_3 = {
#     'category': 'Accommodation',
#     'polarity': 8.0,
#     'columns': ['place_name', 'polarity']
# }

# search_criteria_4 = {
#     'lat': '52.0<x<53.0',
#     'lng': '4.8<x<4.9',
#     'columns': ['place_name', 'lat', 'lng']
# }

# # Perform Searches
# print("\nSearch Results 1:")
# result_1 = search_database(df, search_criteria_1)
# print(result_1)

print("\nSearch Results 2:")
result_2 = search_database(df, search_criteria_2)
print(result_2)

# print("\nSearch Results 3:")
# result_3 = search_database(df, search_criteria_3)
# print(result_3)

# print("\nSearch Results 4:")
# result_4 = search_database(df, search_criteria_4)
# print(result_4)

# Example Usage of Loading Function
if __name__ == "__main__":
    # You can place the search examples here or elsewhere as needed
    pass  # Already executed above


Data successfully loaded from location_data_frames.pkl

Search Results 2:


/var/folders/0d/3y0gvcjx1j74wcywdv3rd8n80000gn/T/ipykernel_10956/637137770.py:154: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = df[mask]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).